In [1]:
import numpy as np
import pandas as pd
import os

from flipside import Flipside
flipside = Flipside("023b83a2-e411-4849-a961-3742c87bc9c9", "https://api-v2.flipsidecrypto.xyz")

We first set the name of the liquidity pool we want to query data from.

In [2]:
pool_name = "USDC-WETH 3000 60"

This write the sql code and start query.

In [3]:
sql = """
SELECT pool_name, tx_hash, action, block_number, block_timestamp, 
       liquidity_provider, nf_token_id,
       amount0_adjusted, token0_symbol, amount0_usd,
       amount1_adjusted, token1_symbol, amount1_usd,
       tick_lower, tick_upper
FROM ethereum.uniswapv3.ez_lp_actions
""" +\
" WHERE pool_name = '" + pool_name + "' " +\
"""
ORDER BY block_number DESC
"""

# Since there is a 30mb rate limit for each page, we need to split the data in seperate pages, we will merge them later
# If we are retrieving a dataset with more columns, we might want to set page_size to a lower number
query_result_set = flipside.query(sql, page_number = 1, page_size = 30000)

Here, we run a loop to merge the pages into one full dataset.

In [4]:
current_page_number = 1
page_size = 30000
# We first set total_pages to 2 (any number larger than current_page_number will do) for the loop to run, we will retrieve the actual total pages in the loop
total_pages = 2

# All the data will be stored in a list called "all_rows"
all_rows = []

while current_page_number <= total_pages:
  results = flipside.get_query_results(
    query_result_set.query_id,
    page_number=current_page_number,
    page_size=page_size
  )

  total_pages = results.page.totalPages
  if results.records:
      all_rows = all_rows + results.records
  
  current_page_number += 1

# Merge all rows to a dataframe
df = pd.DataFrame(all_rows)
df.drop(columns = ['__row_index'], inplace = True)
df.head(10)

,pool_name,tx_hash,action,block_number,block_timestamp,liquidity_provider,nf_token_id,amount0_adjusted,token0_symbol,amount0_usd,amount1_adjusted,token1_symbol,amount1_usd,tick_lower,tick_upper
0,USDC-WETH 3000 60,0xb4df9f76e6da218bd019de8fd92ab150f95ac2b4f368...,DECREASE_LIQUIDITY,17677422,2023-07-12T12:16:11.000Z,0xb2f6244baecd25ac4b404a8ec5643c085d1cb225,NaN,0.000000,USDC,0.000000,0.000000,WETH,0.000000,195960,205440
1,USDC-WETH 3000 60,0xd040ff705cf85b3f7fe552cef8535b851c12ace744fb...,DECREASE_LIQUIDITY,17676309,2023-07-12T08:31:23.000Z,0x236884bb674067c96fcbcc5dff728141e05be99d,426428.0,65.103728,USDC,65.139789,0.022555,WETH,42.605853,195660,209520
2,USDC-WETH 3000 60,0x1721640d0d1014d861e0fccaa668a6425e5a1b3b0f4e...,INCREASE_LIQUIDITY,17676277,2023-07-12T08:24:47.000Z,0xf7f63d1e5cadd293522e62e279b7bc9b7432aca7,536406.0,1226.703857,USDC,1227.383327,0.500000,WETH,944.468734,200280,201660
3,USDC-WETH 3000 60,0x9e7460ffcba97689cc6308aad21eb632b83faaaf6e6c...,DECREASE_LIQUIDITY,17676019,2023-07-12T07:32:11.000Z,0xef027178de9c27d4c141c306aa7a991508f53ee4,349149.0,1.254108,USDC,1.254266,0.001626,WETH,3.075427,-887220,211380
4,USDC-WETH 3000 60,0x4f2f787950fb19fb314c8feb6d05ac75e62d25c25191...,DECREASE_LIQUIDITY,17675505,2023-07-12T05:47:59.000Z,0x554d29160f779adf0a4328597cd33ea1df4d9ee9,482958.0,977.461853,USDC,977.416330,0.400553,WETH,754.601565,198000,204780
5,USDC-WETH 3000 60,0x334331f6266335fbc524d0bb1320e12742da71685dfd...,DECREASE_LIQUIDITY,17675429,2023-07-12T05:32:35.000Z,0xd6810237e16a2df205eb41ca4d28ab1d5cc3e744,NaN,0.000000,USDC,0.000000,0.000000,WETH,0.000000,198000,204780
6,USDC-WETH 3000 60,0x7116d40e11aaaf920e5f0e93acba5c867bae9ecc6786...,DECREASE_LIQUIDITY,17674738,2023-07-12T03:11:23.000Z,0x7dd9fba62875090e90dd83755697f8303043b6c9,161071.0,0.000000,USDC,0.000000,2.013577,WETH,3788.284969,189420,194700
7,USDC-WETH 3000 60,0x7e6a5b32997b2c4df7d04287695f227cda0d5ec36f82...,DECREASE_LIQUIDITY,17674727,2023-07-12T03:09:11.000Z,0x7dd9fba62875090e90dd83755697f8303043b6c9,NaN,0.000000,USDC,0.000000,0.000000,WETH,0.000000,189420,194700
8,USDC-WETH 3000 60,0x1969b7033403155b236b9b1dec9a35c4c0fa39d2b2e2...,DECREASE_LIQUIDITY,17674688,2023-07-12T03:01:23.000Z,0x88513393cb4c3401016694a0fb74765e9b4cb455,NaN,10.436664,USDC,10.436704,0.000000,WETH,0.000000,201060,202260
9,USDC-WETH 3000 60,0x1969b7033403155b236b9b1dec9a35c4c0fa39d2b2e2...,DECREASE_LIQUIDITY,17674688,2023-07-12T03:01:23.000Z,0x88513393cb4c3401016694a0fb74765e9b4cb455,NaN,323.671302,USDC,323.672553,0.162199,WETH,305.155699,197400,204660


Lastly, we want to save the data in our desire repository.

In [5]:
df.to_csv("/Users/zhicong/Dropbox/DEXs/uniswapv3_lp_actions/" + pool_name + ".csv", index=False)